# 目录
1. [分词算法](#一、分词算法)
    1. [基于匹配规则的算法](#1、基于匹配规则的算法)
    2. [基于概率统计的方法(LM,HMM,CRF..)](#2、基于概率统计的方法(LM,HMM,CRF..))
        1. [维特比算法](#维特比(viterbi)算法:)
        2. [HMM实现](#基于HMM模型的分词算法)
2. [分词工具jieba](#二、分词工具jieba)    
3. [TODO](#TODO：)

# 一、分词算法

### 1、基于匹配规则的算法
- 最大匹配算法(Max Matching)，按具体实现方式，分为前向、逆向、双向。
- 给定词典 V，根据词典中单词长度的分布决定一个最大匹配长度，如 max_len=5，作为算法的参数。
- 前向最大匹配算法示意图：
    
<img style="float: middle;" src="../images/最大前向匹配.png" width="60%">

#### 逆向最大匹配算法实现
- 时间复杂度：$O(n * max\_len)$

In [6]:
class InverseMaxMatching:
    def __init__(self, dict_path):
        self.dictionary = set()
        self.max_length = 0
        with open(dict_path, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                self.dictionary.add(line)
                if len(line) > self.max_length:
                    self.max_length = len(line)

    def cut(self, text):
        result = []
        index = len(text)
        while index > 0:
            word = None
            for size in range(self.max_length, 0, -1):
                if index - size < 0:
                    continue
                piece = text[(index - size):index]
                if piece in self.dictionary:
                    word = piece
                    result.append(word)
                    index -= size
                    break
            if word is None:
                index -= 1
        return result[::-1]

In [7]:
dict_path = 'datasets/imm_dic.utf8'
print('词典如下：')
with open(dict_path, 'r', encoding='utf-8') as f:
    for line in f:
        if line: print(line.strip())

词典如下：
深度
学习
深度学习
很
强大
很强大


In [8]:
text = '深度学习很强大'
tokenizer = InverseMaxMatching(dict_path)
tokenizer.cut(text)

['深度学习', '很强大']

#### 匹配算法的缺点如下:
- 只能得到局部最优
- 效率低，时间复杂度为 $O(max\_len * n)$
- 不能细分
- 不能考虑歧义

### 2、基于概率统计的方法(LM,HMM,CRF..)
- 给定**语言模型(LM)**，如 `Unigram-language model`，统计每个单词在语料库中出现的概率，如单词“经常”出现的概率：$P(经常) = \frac{"经常"出现的次数}{语料库中单词总数}=\frac{500}{10^6}$，其值通常为极小值。
    
- 生成句子所有可能的分割，如 s1= “经常/有/意见/分歧“，s2=“经常/有意见/分歧“ 两种分割方案，选择出现概率较大的那一种：
   
$$P(s1)=P(经常)\cdot P(有)\cdot P(意见)\cdot P(分歧)$$    
$$P(s2)=P(经常)\cdot P(有意见)\cdot P(分歧)$$
- 因为每个单词的出现概率比较小，可能导致最终分割方案的概率极小，超出计算机处理范围(underfloat)，通常采用对数概率：
$$logP(s1)=logP(经常)+logP(有)+logP(意见)+logP(分歧)$$

#### [维特比(`viterbi`)算法:](#目录)
- 每个句子的分割方案可能很多，导致计算的复杂度很高；可以将其转化为求解最短路径的问题，利用维特比(`viterbi`)算法快速求解。
$$f(m)=min\bigg[f(i)+dist(i,m)\bigg]\quad{i\in incomings\ of\ m}$$
    
- 示例：对 “经常有意见分歧” 进行分词

词典：|经常|有|有意见|意见|分歧|见|见分歧|分|其它
-|-|-|-|-|-|-|-|-|- 
概率：|0.1 |0.05|0.1| 0.2|0.2|0.05|0.05|0.1|2.1e-9   
 -log(x)：|2.3|3| 2.3| 1.6|1.6 |3 |3|2.3|20

<img src="../images/维特比算法.png" alt="维特比算法" width="60%"/>

- 利用动态规划，可得出最短路径
$$
\begin{align*}
f(8)&=min\{f(7)+dist(7,8);f(6)+dist(6,8);f(5)+dist(5,8)\}\\
f(7)&=min\{f(6)+dist(6,7)\}\\
f(6)&=min\{f(5)+dist(5,6);f(4)+dist(4,6);f(3)+dist(3,6)\}\\
f(5)&=min\{f(4)+dist(4,5)\}\\
f(4)&=min\{f(3)+dist(3,4)\}\\
f(3)&=min\{f(2)+dist(2,3);f(1)+dist(1,3)\}\\
f(2)&=min\{f(1)+dist(1,2)\}\\
f(1)&=0\\
\end{align*}
$$
- **分词可以认为是已经解决了的问题。**

#### [基于HMM模型的分词算法](#目录)

In [10]:
class HMM(object):
    def __init__(self):
        import os

        # 主要是用于存取算法中间结果，不用每次都训练模型
        self.model_file = 'models/hmm_model.pkl'

        # 状态值集合
        self.state_list = ['B', 'M', 'E', 'S']
        # 参数加载,用于判断是否需要重新加载model_file
        self.load_para = False

    # 用于加载已计算的中间结果，当需要重新训练时，需初始化清空结果
    def try_load_model(self, trained):
        if trained:
            import pickle
            with open(self.model_file, 'rb') as f:
                self.A_dic = pickle.load(f)
                self.B_dic = pickle.load(f)
                self.Pi_dic = pickle.load(f)
                self.load_para = True

        else:
            # 状态转移概率（状态->状态的条件概率）
            self.A_dic = {}
            # 发射概率（状态->词语的条件概率）
            self.B_dic = {}
            # 状态的初始概率
            self.Pi_dic = {}
            self.load_para = False

    # 计算转移概率、发射概率以及初始概率
    def train(self, path):

        # 重置几个概率矩阵
        self.try_load_model(False)

        # 统计状态出现次数，求p(o)
        Count_dic = {}

        # 初始化参数
        def init_parameters():
            for state in self.state_list:
                self.A_dic[state] = {s: 0.0 for s in self.state_list}
                self.Pi_dic[state] = 0.0
                self.B_dic[state] = {}

                Count_dic[state] = 0

        def makeLabel(text):
            out_text = []
            if len(text) == 1:
                out_text.append('S')
            else:
                out_text += ['B'] + ['M'] * (len(text) - 2) + ['E']

            return out_text

        init_parameters()
        line_num = -1
        # 观察者集合，主要是字以及标点等
        words = set()
        with open(path, encoding='utf8') as f:
            for line in f:
                line_num += 1

                line = line.strip()
                if not line:
                    continue

                word_list = [i for i in line if i != ' ']
                words |= set(word_list)  # 更新字的集合

                linelist = line.split()

                line_state = []
                for w in linelist:
                    line_state.extend(makeLabel(w))

                assert len(word_list) == len(line_state)

                for k, v in enumerate(line_state):
                    Count_dic[v] += 1
                    if k == 0:
                        self.Pi_dic[v] += 1  # 每个句子的第一个字的状态，用于计算初始状态概率
                    else:
                        self.A_dic[line_state[k - 1]][v] += 1  # 计算转移概率
                        self.B_dic[line_state[k]][word_list[k]] = \
                            self.B_dic[line_state[k]].get(word_list[k], 0) + 1.0  # 计算发射概率

        self.Pi_dic = {k: v * 1.0 / line_num for k, v in self.Pi_dic.items()}
        self.A_dic = {
            k: {k1: v1 / Count_dic[k]
                for k1, v1 in v.items()}
            for k, v in self.A_dic.items()
        }
        #加1平滑
        self.B_dic = {
            k: {k1: (v1 + 1) / Count_dic[k]
                for k1, v1 in v.items()}
            for k, v in self.B_dic.items()
        }
        #序列化
        import pickle
        with open(self.model_file, 'wb') as f:
            pickle.dump(self.A_dic, f)
            pickle.dump(self.B_dic, f)
            pickle.dump(self.Pi_dic, f)

        return self

    def viterbi(self, text, states, start_p, trans_p, emit_p):
        V = [{}]
        path = {}
        for y in states:
            V[0][y] = start_p[y] * emit_p[y].get(text[0], 0)
            path[y] = [y]
        for t in range(1, len(text)):
            V.append({})
            newpath = {}

            #检验训练的发射概率矩阵中是否有该字
            neverSeen = text[t] not in emit_p['S'].keys() and \
                text[t] not in emit_p['M'].keys() and \
                text[t] not in emit_p['E'].keys() and \
                text[t] not in emit_p['B'].keys()
            for y in states:
                emitP = emit_p[y].get(text[t],
                                      0) if not neverSeen else 1.0  #设置未知字单独成词
                (prob, state) = max([
                    (V[t - 1][y0] * trans_p[y0].get(y, 0) * emitP, y0)
                    for y0 in states if V[t - 1][y0] > 0
                ])
                V[t][y] = prob
                newpath[y] = path[state] + [y]
            path = newpath

        if emit_p['M'].get(text[-1], 0) > emit_p['S'].get(text[-1], 0):
            (prob, state) = max([(V[len(text) - 1][y], y) for y in ('E', 'M')])
        else:
            (prob, state) = max([(V[len(text) - 1][y], y) for y in states])

        return (prob, path[state])

    def cut(self, text):
        import os
        if not self.load_para:
            self.try_load_model(os.path.exists(self.model_file))
        prob, pos_list = self.viterbi(text, self.state_list, self.Pi_dic,
                                      self.A_dic, self.B_dic)
        begin, next = 0, 0
        for i, char in enumerate(text):
            pos = pos_list[i]
            if pos == 'B':
                begin = i
            elif pos == 'E':
                yield text[begin:i + 1]
                next = i + 1
            elif pos == 'S':
                yield char
                next = i + 1
        if next < len(text):
            yield text[next:]

In [11]:
hmm = HMM()
hmm.train('datasets/trainCorpus.txt_utf8')

In [12]:
text = '这是一个非常棒的方案！'
res = hmm.cut(text)
print(text)
print(str(list(res)))

这是一个非常棒的方案！
['这是', '一个', '非常', '棒', '的', '方案', '！']


# [二、分词工具jieba](#目录)

**分词**

In [14]:
import jieba

In [19]:
sent = '中文分词是文本处理不可或缺的一步！'
seg_list = jieba.cut(sent, cut_all=True)
print('全模式：', ' ' .join(seg_list)) 

全模式： 中文 分词 是 文本 文本处理 本处 处理 不可 不可或缺 或缺 的 一步 ！


In [16]:
seg_list = jieba.cut(sent, cut_all=False)
print('精确模式：', ' '.join(seg_list)) 

精确模式： 中文 分词 是 文本处理 不可或缺 的 一步 ！


In [17]:
seg_list = jieba.cut(sent)  
print('默认精确模式：', ' '.join(seg_list))

默认精确模式： 中文 分词 是 文本处理 不可或缺 的 一步 ！


In [18]:
seg_list = jieba.cut_for_search(sent)  
print('搜索引擎模式：', ' '.join(seg_list))

搜索引擎模式： 中文 分词 是 文本 本处 处理 文本处理 不可 或缺 不可或缺 的 一步 ！


**带词性标注的分词**

In [20]:
import jieba.posseg as psg

sent = '中文分词是文本处理不可或缺的一步！'
seg_list = psg.cut(sent)

print(''.join(['{0}[{1}]'.format(w, t) for w, t in seg_list]))

中文[nz]分词[n]是[v]文本处理[n]不可或缺[l]的[uj]一步[m]！[x]


**自定义词典**

In [21]:
print('自定义词典内容：')
with open('datasets/user_dict.utf8', 'r', encoding='utf-8') as f:
    for line in f:
        if line:
            print(line.strip())

自定义词典内容：
大波浪 10
jieba分词 n
金融词典 7


In [22]:
sent = 'jieba分词非常好用，可以自定义金融词典！'

seg_list = jieba.cut(sent)
print('系统词典:', ' '.join(seg_list))

系统词典: jieba 分词 非常 好用 ， 可以 自定义 金融 词典 ！


In [23]:
jieba.load_userdict('./datasets/user_dict.utf8')
seg_list = jieba.cut(sent)
print('自定义词典:', ' '.join(seg_list))

自定义词典: jieba分词 非常 好用 ， 可以 自定义 金融词典 ！


**调节特定词词频**

In [24]:
sent = '好丑的证件照片'
print(' '.join(jieba.cut(sent, HMM=False)))

jieba.suggest_freq(('证件照片'), True)
print(' '.join(jieba.cut(sent, HMM=False)))

好丑 的 证件 照片
好丑 的 证件照片


**提取关键词**

In [25]:
from jieba import analyse

content = """
自然语言处理（NLP）是计算机科学，人工智能，语言学关注计算机和人类（自然）语言之间的相互作用的领域。
因此，自然语言处理是与人机交互的领域有关的。在自然语言处理面临很多挑战，包括自然语言理解，因此，自然语言处理涉及人机交互的面积。
在NLP诸多挑战涉及自然语言理解，即计算机源于人为或自然语言输入的意思，和其他涉及到自然语言生成。
"""

# 加载自定义idf词典
analyse.set_idf_path('datasets/idf.txt.big')

# 加载停用词典
analyse.set_stop_words('datasets/stop_words.utf8')

# 第一个参数：待提取关键词的文本
# 第二个参数：返回关键词的数量，重要性从高到低排序
# 第三个参数：是否同时返回每个关键词的权重
# 第四个参数：词性过滤，为空表示不过滤，若提供则仅返回符合词性要求的关键词
keywords = analyse.extract_tags(content, topK=10, withWeight=True, allowPOS=())

for item in keywords:
    print(item[0], item[1])

自然语言 2.0790900005043476
NLP 0.5197725001260869
计算机 0.5197725001260869
领域 0.5197725001260869
人机交互 0.5197725001260869
挑战 0.5197725001260869
理解 0.5197725001260869
处理 0.4705091875965217
涉及 0.3839134341652174
人工智能 0.25988625006304344


**TextRank**

In [26]:
# 第一个参数：待提取关键词的文本
# 第二个参数：返回关键词的数量，重要性从高到低排序
# 第三个参数：是否同时返回每个关键词的权重
# 第四个参数：词性过滤，为空表示过滤所有，与TF—IDF不一样！
keywords = jieba.analyse.textrank(content,
                                  topK=10,
                                  withWeight=True,
                                  allowPOS=('ns', 'n', 'vn', 'v'))
for item in keywords:
    # 分别为关键词和相应的权重
    print(item[0], item[1])

涉及 1.0
计算机 0.9618169519358478
处理 0.8124660402732825
理解 0.7885898958379202
挑战 0.7833575495518058
人机交互 0.7343470452632993
语言学 0.727536034596871
人类 0.6290562193534068
人工智能 0.5809911385488661
关注 0.577881611632419


#### 处理文本文件

In [27]:
def get_content(path):
    with open(path, 'r', encoding='gbk', errors='ignore') as f:
        content = ''
        for line in f:
            line = line.strip()
            content += line
        return content

In [28]:
def get_tf(words, topK=10):
    tf_dic = {}
    for word in words:
        tf_dic[word] = tf_dic.get(word, 0) + 1
    return sorted(tf_dic.items(), key=lambda x: x[1], reverse=True)[:topK]

In [29]:
def stop_words(path):
    with open(path, 'r', encoding='utf-8', errors='ignore') as f:
        return [l.strip() for l in f]

In [30]:
import jieba
import jieba.posseg as psg
import random
import glob

files = glob.glob('datasets/news/C000008/*.txt')
corpus = [get_content(path) for path in files[:5]]

sample = random.choice(corpus)

split_words = [
    x for x in jieba.cut(sample)
    if not x.isspace() and x not in stop_words('./datasets/stop_words.utf8')
]
print("样本：", sample)

样本： 证券通：Ｇ 皖 通 车流量稳定增长 大股东可能注入优质资产（推荐）股票评级☆☆☆☆☆深度报告 权威内参 来自“证券通”www.KL178.comG皖通(行情,论坛)（600012）05年业绩增长迅速，计重收费方式下公司单车收入的大幅增长及合宁线的改造完成带来了利润的超速成长；在计重收费下的一次性惩罚性收入效应逐渐削弱的情况下，公司将通过内生车流量的增长，主营成本的下降（稳定的道路养护成本以及直线折旧法）的方式使得业绩在目前基础上依旧有望获得持续提升。考虑到公司车流量稳定增长以及大股东注入优质资产的可能，另外，根据股改协议未来数年高分红比率将能实现（未来数年股息收益率将在6％以上），参考A股公路定价水平，06年12-14倍PE定价是能够被市场接受的，合理价格区间是5.4-6.3元，目前股价存在一定低估，仍有14％－33％上升空间。因此给予投资评级为“推荐”。更多详情免费咨询021*64690729或登录www.KL178.com（证券通），资深行业研究员为您提供客观、深度、超前的投资信息。本文版权为“证券通”独家拥有，任何单位和个人不得复制、转发以及用于业务经营，违者将追究其法律责任。评级说明★★★★★ 坚决规避☆★★★★ 适当减仓☆☆★★★ 风险较大☆☆☆★★ 审慎推荐☆☆☆☆★ 首次推荐☆☆☆☆☆ 强烈推荐作者声明：在本机构、本人所知情的范围内,本机构、本人以及财产上的利害关系人与所述文章内容没有利害关系。本版文章纯属个人观点,仅供参考,文责自负。读者据此入市,风险自担。


In [32]:
print("分词结果：", '|'.join(split_words))

分词结果： 证券|通|Ｇ|皖|通|车流量|稳定增长|股东|可能|注入|优质|资产|推荐|股票|评级|☆|☆|☆|☆|☆|深度|报告|权威|内参|证券|通|www|KL178|comG|皖通|行情|论坛|600012|05|年|业绩|增长|迅速|计重|收费|方式|公司|单车|收入|大幅|增长|及合宁线|改造|完成|带来|利润|超速|成长|计重|收费|一次性|惩罚性|收入|效应|逐渐|削弱|情况|公司|内生|车流量|增长|主营|成本|下降|稳定|道路|养护|成本|直线|折旧|法|方式|业绩|目前|基础|依旧|有望|获得|持续|提升|考虑|公司|车流量|稳定增长|股东|注入|优质|资产|可能|股改|协议|未来|数年|高|分红|比率|实现|未来|数年|股息|收益率|％|参考|A股|公路|定价|水平|06|年|12|14|倍|PE|定价|能够|市场|接受|合理|价格|区间|5.4|6.3|元|目前|股价|存在|一定|低估|14|％|－|33|％|上升|空间|给予|投资|评级|推荐|更|详情|免费|咨询|021|64690729|登录|www|KL178|com|证券|通|资深|行业|研究员|提供|客观|深度|超前|投资|信息|本文|版权|证券|通|独家|拥有|单位|个人|复制|转发|用于|业务|经营|违者|追究其|法律责任|评级|说明|★|★|★|★|★|坚决|规避|☆|★|★|★|★|适当|减仓|☆|☆|★|★|★|风险|较大|☆|☆|☆|★|★|审慎|推荐|☆|☆|☆|☆|★|首次|推荐|☆|☆|☆|☆|☆|强烈推荐|作者|声明|机构|知情|范围|机构|财产|利害|关系人|所述|文章内容|没有|利害关系|本版|文章|纯属|个人观点|仅供参考|文责自负|读者|入市|风险|自担


In [33]:
print("样本的topK(10)词：", str(get_tf(split_words)))

样本的topK(10)词： [('☆', 20), ('★', 15), ('通', 5), ('证券', 4), ('推荐', 4), ('车流量', 3), ('评级', 3), ('增长', 3), ('公司', 3), ('％', 3)]


In [34]:
split_words = [
    w for w, t in psg.cut(sample) if not w.isspace()
    and w not in stop_words('datasets/stop_words.utf8') and t.startswith('n')
]
print('样本之一：' + sample)

样本之一：证券通：Ｇ 皖 通 车流量稳定增长 大股东可能注入优质资产（推荐）股票评级☆☆☆☆☆深度报告 权威内参 来自“证券通”www.KL178.comG皖通(行情,论坛)（600012）05年业绩增长迅速，计重收费方式下公司单车收入的大幅增长及合宁线的改造完成带来了利润的超速成长；在计重收费下的一次性惩罚性收入效应逐渐削弱的情况下，公司将通过内生车流量的增长，主营成本的下降（稳定的道路养护成本以及直线折旧法）的方式使得业绩在目前基础上依旧有望获得持续提升。考虑到公司车流量稳定增长以及大股东注入优质资产的可能，另外，根据股改协议未来数年高分红比率将能实现（未来数年股息收益率将在6％以上），参考A股公路定价水平，06年12-14倍PE定价是能够被市场接受的，合理价格区间是5.4-6.3元，目前股价存在一定低估，仍有14％－33％上升空间。因此给予投资评级为“推荐”。更多详情免费咨询021*64690729或登录www.KL178.com（证券通），资深行业研究员为您提供客观、深度、超前的投资信息。本文版权为“证券通”独家拥有，任何单位和个人不得复制、转发以及用于业务经营，违者将追究其法律责任。评级说明★★★★★ 坚决规避☆★★★★ 适当减仓☆☆★★★ 风险较大☆☆☆★★ 审慎推荐☆☆☆☆★ 首次推荐☆☆☆☆☆ 强烈推荐作者声明：在本机构、本人所知情的范围内,本机构、本人以及财产上的利害关系人与所述文章内容没有利害关系。本版文章纯属个人观点,仅供参考,文责自负。读者据此入市,风险自担。


In [35]:
print('样本分词效果：' + '/ '.join(split_words))

样本分词效果：证券/ 车流量/ 稳定增长/ 股东/ 优质/ 资产/ 股票/ 深度/ 报告/ 权威/ 内参/ 证券/ 皖通/ 行情/ 论坛/ 业绩/ 计重/ 收费/ 方式/ 公司/ 单车/ 合宁线/ 利润/ 计重/ 收费/ 惩罚性/ 效应/ 情况/ 公司/ 内生/ 车流量/ 成本/ 道路/ 成本/ 直线/ 方式/ 业绩/ 基础/ 公司/ 车流量/ 稳定增长/ 股东/ 优质/ 资产/ 协议/ 比率/ 股息/ 收益率/ A股/ 公路/ 定价/ 水平/ 定价/ 市场/ 价格/ 区间/ 股价/ 空间/ 证券/ 资深/ 行业/ 研究员/ 客观/ 深度/ 信息/ 版权/ 证券/ 独家/ 单位/ 个人/ 业务/ 违者/ 追究其/ 法律责任/ 风险/ 作者/ 声明/ 机构/ 知情/ 范围/ 机构/ 财产/ 关系人/ 文章内容/ 利害关系/ 文章/ 纯属/ 个人观点/ 读者/ 入市/ 风险


In [36]:
print('样本的topK（10）词：'+str(get_tf(split_words)))

样本的topK（10）词：[('证券', 4), ('车流量', 3), ('公司', 3), ('稳定增长', 2), ('股东', 2), ('优质', 2), ('资产', 2), ('深度', 2), ('业绩', 2), ('计重', 2)]


### [TODO：](#目录)
- 基于语言模型和维特比算法的分词实现
- 基于 HMM 模型的理解